In [250]:
import re
import json 
import pickle
from datasets import load_dataset 
from langchain.schema import HumanMessage, AIMessage

# input_path = "data/runs/agentinstruct_db_syn_v4_filtered_v2-sharegpt-dbbench-dev.jsonl"
# output_path = "data/runs/agentinstruct_db_syn_v4_filtered_v2-sharegpt-dbbench-dev-to_view.pickle"
input_path = r"C:\Users\harsh\Documents\crypto-LLM-exps\synchaev\data\dbbench\agentinstruct_db.json"
output_path = r"C:\Users\harsh\Documents\crypto-LLM-exps\synchaev\data\dbbench\agentinstruct_db-to_view.pickle"

In [251]:
if "jsonl" in input_path: 
    runs_list = []

    with open(input_path, 'r') as file:
        for line in file:
            runs_list.append(json.loads(line))
else:
    with open(input_path, "r") as f:
        runs_list = json.load(f)

#### AgentBench runs

OS Interaction

In [116]:
runs_to_view_list = {"agents": [], 'environments': []}

for runs in runs_list:
    agent_conv, env_conv = [], []
    for i, msg in enumerate(runs["output"]["history"]):
        if msg["role"] == "agent":    
            agent_conv.append(AIMessage(content=msg["content"], additional_kwargs={}, example=False))
        if msg["role"] == "user":
            agent_conv.append(HumanMessage(content=msg["content"], additional_kwargs={}, example=False))
            if i!=0:
                env_conv.append(HumanMessage(content="", additional_kwargs={}, example=False))
                env_conv.append(HumanMessage(content=msg["content"], additional_kwargs={}, example=False))
    runs_to_view_list["agents"].append(agent_conv)
    runs_to_view_list["environments"].append(env_conv)

In [44]:
with open(output_path, 'wb') as file:
    pickle.dump(runs_to_view_list, file)

DBBench 

In [213]:
runs_to_view_list = {"agents": [], 'environments': []}

for runs in runs_list:
    agent_conv, env_conv = [], []
    for i, msg in enumerate(runs["output"]["history"]):
        if msg["role"] == "agent":    
            agent_conv.append(AIMessage(content=msg["content"], additional_kwargs={}, example=False))
            sql_block = re.search(r"```sql(.*?)```", msg["content"], re.DOTALL)
            if sql_block:
                sql_code = sql_block.group(1).strip()
                env_conv.append(HumanMessage(content=f"```sql\n{sql_code}\n```", additional_kwargs={}, example=False))
        if msg["role"] == "user":
            agent_conv.append(HumanMessage(content=msg["content"], additional_kwargs={}, example=False))
            if i!=0 and i!=2:
                env_conv.append(HumanMessage(content=msg["content"], additional_kwargs={}, example=False))
    runs_to_view_list["agents"].append(agent_conv)
    runs_to_view_list["environments"].append(env_conv)

KeyError: 'output'

In [206]:
with open(output_path, 'wb') as file:
    pickle.dump(runs_to_view_list, file)

#### Synthetic Data

DBBench

In [252]:
runs_to_view_list = {"agents": [], 'environments': []}

for runs in runs_list:
    agent_conv, env_conv = [], []
    for i, msg in enumerate(runs["conversations"]):
        if i==0 or i==2:
            agent_conv.append(HumanMessage(content=msg["value"], additional_kwargs={}, example=False))
        if msg["from"] == "gpt":    
            agent_conv.append(AIMessage(content=msg["value"], additional_kwargs={}, example=False))
            # sql_block = re.search(r"```sql(.*?)```", msg["value"], re.DOTALL)
            # if sql_block:
            #     sql_code = sql_block.group(1).strip()
            #     env_conv.append(HumanMessage(content=f"```sql\n{sql_code}\n```", additional_kwargs={}, example=False))
        if msg["from"] == "human":
            # agent_conv.append(HumanMessage(content=msg["value"], additional_kwargs={}, example=False))
            if i!=0 and i!=2:
                env_conv.append(HumanMessage(content=msg["value"], additional_kwargs={}, example=False))
    runs_to_view_list["agents"].append(agent_conv)
    runs_to_view_list["environments"].append(env_conv)

In [253]:
with open(output_path, 'wb') as file:
    pickle.dump(runs_to_view_list, file)